In [4]:
%%writefile scripts/simpleCoinbase.html
<!doctype>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/web3@1.2.5/dist/web3.min.js"></script> 
<script type="text/javascript">
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

    function displayCoinbase() {
        
        document.getElementById('web3ver').innerText = 'web3 version 1.2.X';
        web3.eth.getCoinbase(function(err, coinbase){
            document.getElementById('web3coinbase').innerText = 'coinbase: ' + coinbase;
        })
    }
</script>
</head>
<body>
    <h1>Hello</h1>
    <button type="button" onClick="displayCoinbase();">Display Coinbase</button>
    <div></div>
    <div id="web3ver"></div>
    <div id="web3coinbase"></div>
</body>
</html>

Overwriting scripts/simpleCoinbase.html


### 파이썬 서버 실행
python -m http.server 8045  
localhost:8045/scripts

## sayHello 배포 및 사용

In [1]:
!node src/HelloDeploy.js

accounts: 0x03D8F3F2a4Ca98C225168DF094B96347B6Eb8A70
null 0x89f95dd1274539bc49a7f74824ffd3f921ce0b4799a2b66c562b4191937a0f57
0xBD906283B8f8f036383a7E0aA4CADF5d569De8a3


In [2]:

%%writefile scripts/hello.html
<!doctype>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/web3@1.2.5/dist/web3.min.js"></script>
<script type="text/javascript">

    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

    function displayHello() {

        document.getElementById('web3ver').innerText = 'web3 version 1.2.x';
        web3.eth.getCoinbase().then(function(coin) {
            document.getElementById('web3coinbase').innerText = 'coinbase: ' + coin;
            });
        
        var shelloContract = new web3.eth.Contract([{"constant":true,"inputs":[],"name":"sayHello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"}],
                                      "0xBD906283B8f8f036383a7E0aA4CADF5d569De8a3");
        
        shelloContract.methods.sayHello().call().then(function(str) {
            document.getElementById('sayhello').innerText = "The message from blockchain: " + str;
        });
        
    }
</script>
</head>
<body>
    <h1>Hello</h1>
    <button type="button" onClick="displayHello();">Display Hello</button>
    <div></div>
    <div id="sayhello"></div>
    <div id="web3ver"></div>
    <div id="web3coinbase"></div>
</body>
</html>

Overwriting scripts/hello.html


## greeter

### 1. 컨트랙 개발

In [2]:
%%writefile src/greeter.sol
pragma solidity ^0.6.0;

contract Greeter {
    string greeting;
    constructor() public {
        greeting='Hello';
    }
    function setGreeting(string memory _greeting) public {
        greeting = _greeting;
    }
    function greet() view public returns (string memory) {
        return greeting;
    }
}

Writing src/greeter.sol


### 2. 컴파일

In [3]:
!solc --abi src/greeter.sol


======= src/greeter.sol:Greeter =======
Contract JSON ABI
[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"greet","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"string","name":"_greeting","type":"string"}],"name":"setGreeting","outputs":[],"stateMutability":"nonpayable","type":"function"}]


In [4]:
!solc --bin src/greeter.sol


======= src/greeter.sol:Greeter =======
Binary:
608060405234801561001057600080fd5b506040518060400160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005c929190610062565b50610107565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a357805160ff19168380011785556100d1565b828001600101855582156100d1579182015b828111156100d05782518255916020019190600101906100b5565b5b5090506100de91906100e2565b5090565b61010491905b808211156101005760008160009055506001016100e8565b5090565b90565b610310806101166000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063a41368621461003b578063cfae3217146100f6575b600080fd5b6100f46004803603602081101561005157600080fd5b810190808035906020019064010000000081111561006e57600080fd5b82018360208201111561008057600080fd5b803590602001918460018302840111640100000000831117156100a257600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602

3. 배포 web3

In [3]:
%%writefile src/greeterDeploy.js

var Web3=require('web3');
var web3;

if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abiArray=[{"constant":false,"inputs":[{"name":"_greeting","type":"string"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="608060405234801561001057600080fd5b506040518060400160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005c929190610062565b50610107565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a357805160ff19168380011785556100d1565b828001600101855582156100d1579182015b828111156100d05782518255916020019190600101906100b5565b5b5090506100de91906100e2565b5090565b61010491905b808211156101005760008160009055506001016100e8565b5090565b90565b610310806101166000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063a41368621461003b578063cfae3217146100f6575b600080fd5b6100f46004803603602081101561005157600080fd5b810190808035906020019064010000000081111561006e57600080fd5b82018360208201111561008057600080fd5b803590602001918460018302840111640100000000831117156100a257600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290505050610179565b005b6100fe610193565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101561013e578082015181840152602081019050610123565b50505050905090810190601f16801561016b5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b806000908051906020019061018f929190610235565b5050565b606060008054600181600116156101000203166002900480601f01602080910402602001604051908101604052809291908181526020018280546001816001161561010002031660029004801561022b5780601f106102005761010080835404028352916020019161022b565b820191906000526020600020905b81548152906001019060200180831161020e57829003601f168201915b5050505050905090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061027657805160ff19168380011785556102a4565b828001600101855582156102a4579182015b828111156102a3578251825591602001919060010190610288565b5b5090506102b191906102b5565b5090565b6102d791905b808211156102d35760008160009055506001016102bb565b5090565b9056fea26469706673582212209b267c7ef3dcf37f2dffcbd963990eb71ece19f939b74800737072b119dfa18464736f6c63430006040033";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');

_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x03D8F3F2a4Ca98C225168DF094B96347B6Eb8A70", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/greeterDeploy.js


In [4]:
!node src/greeterDeploy.js

0x1E121b89C8c892432f8B3b2966A34c8a906b8077


### 4. 사용

In [7]:
%%writefile src/greeterUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray=[{"constant":false,"inputs":[{"name":"_greeting","type":"string"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var greeter = new web3.eth.Contract(_abiArray,"0x1E121b89C8c892432f8B3b2966A34c8a906b8077");
greeter.methods.greet().call().then(function(value) {console.log(value);});
greeter.methods.setGreeting("Hello SMU").send({from:"0x03D8F3F2a4Ca98C225168DF094B96347B6Eb8A70",gas:100000});
greeter.methods.greet().call().then(function(value) {console.log(value);});

Overwriting src/greeterUse.js


In [55]:
!node src/greeterUse.js

Hello SMU
Hello SMU


## Counter (계수기)

### 1. 컨트랙 개발

In [17]:
%%writefile src/Counter.sol
pragma solidity ^0.6.0;

contract Counter {
    uint256 counter = 0;
    function add() public {
        counter++;
    }
    function subtract() public {
        counter--;
    }
    function getCounter() public view returns (uint256) {
        return counter;
    }
}

Writing src/Counter.sol


### 2. 컴파일

In [18]:
!solc --abi --bin --gas src/Counter.sol


======= src/Counter.sol:Counter =======
Gas estimation:
construction:
   5099 + 42200 = 47299
external:
   add():	20959
   getCounter():	1035
   subtract():	20984
Binary:
60806040526000805534801561001457600080fd5b5060d3806100236000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c80634f2be91f1460415780636deebae31460495780638ada066e146051575b600080fd5b6047606d565b005b604f6080565b005b60576094565b6040518082815260200191505060405180910390f35b6000808154809291906001019190505550565b600080815480929190600190039190505550565b6000805490509056fea264697066735822122080d9d70ff6516aec61cfd97f52f17805751623584c3b022dc487fbfbf3efacde64736f6c63430006040033
Contract JSON ABI
[{"inputs":[],"name":"add","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"subtract","outputs":[],"stateMutability":"nonpayable","type":"f

### 3. 배포

In [12]:
%%writefile src/counterDeploy.js

var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

//solc 0.6.1
var _abiArray=[{"inputs":[],"name":"add","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"subtract","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin="60806040526000805534801561001457600080fd5b5060d3806100236000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c80634f2be91f1460415780636deebae31460495780638ada066e146051575b600080fd5b6047606d565b005b604f6080565b005b60576094565b6040518082815260200191505060405180910390f35b6000808154809291906001019190505550565b600080815480929190600190039190505550565b6000805490509056fea2646970667358221220a19d7e0374295c3c6dd75807d6b2bb20a12deb6f736a4ad98c0065f0d9d4bf5764736f6c63430006010033";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x8d53220a105a9c50E7E6b1c05103D2dF943AB0e2", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/counterDeploy.js


In [13]:
!node src/counterDeploy.js

0x463299E1db2B02b4BA2EA2C7aB8c83d36967DC72


### 4. 사용

In [9]:
%%writefile src/counterUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var abi =[{"constant":false,"inputs":[],"name":"add","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"subtract","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getCounter","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}];
var addr = "0x36d072eCea5AF6274EfFb6dBCf94a2ff79558EA3";
var counter = new web3.eth.Contract(abi,addr);

counter.methods.getCounter().call().then(function(str) {console.log(str);});
//counter.methods.subtract().send({from:"0x988c7734a7D3f5e57d45870a83aE6CdC4d0Ad9b5",gas:100000});
//counter.methods.add().send({from:"0xF9e3D4bCDAA6E04a8a599Bf87EAb7f7d972D3142",gas:100000});
counter.methods.getCounter().call().then(function(str) {console.log(str);});

Overwriting src/counterUse.js


In [10]:
!node src/counterUse.js

(node:9240) UnhandledPromiseRejectionWarning: Error: Invalid JSON RPC response: ""
    at Object.InvalidResponse (C:\Users\yoonho\Code\201910812\node_modules\web3-core-helpers\src\errors.js:42:16)
    at XMLHttpRequest.request.onreadystatechange (C:\Users\yoonho\Code\201910812\node_modules\web3-providers-http\src\index.js:106:32)
    at XMLHttpRequestEventTarget.dispatchEvent (C:\Users\yoonho\Code\201910812\node_modules\xhr2-cookies\dist\xml-http-request-event-target.js:34:22)
    at XMLHttpRequest._setReadyState (C:\Users\yoonho\Code\201910812\node_modules\xhr2-cookies\dist\xml-http-request.js:208:14)
    at XMLHttpRequest._onHttpRequestError (C:\Users\yoonho\Code\201910812\node_modules\xhr2-cookies\dist\xml-http-request.js:349:14)
    at ClientRequest.<anonymous> (C:\Users\yoonho\Code\201910812\node_modules\xhr2-cookies\dist\xml-http-request.js:252:61)
    at ClientRequest.emit (events.js:315:20)
    at Socket.socketErrorListener (_http_client.js:469:9)
    at Socket.emit (events.js:

## Timer

### 1. 컨트랙 개발

In [22]:
%%writefile src/Timer.sol
pragma solidity ^0.6.0;
contract Timer {
    
    uint256 startTime;
    
    function start() public {
        startTime=now;
    }
    function timePassed() public view returns(uint256) {
        return now-startTime;
    }
    function getStart() public view returns(uint256) {
        return startTime;
    }
    function getNow() view public returns(uint) {
        return now;
    }
}

Overwriting src/Timer.sol


### 2. 컴파일
컴파일 결과를 js파일로 저장

In [23]:
!echo | set /p="exports._compiled=" > src\Timer.js
!solc --optimize --combined-json abi,bin,interface src/Timer.sol >> src\Timer.js

In [24]:
!type src\Timer.js

exports._compiled={"contracts":{"src/Timer.sol:Timer":{"abi":"[{\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getStart\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5060c58061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060465760003560e01c8063b445425314604b578063bbe4fd50146063578063be9a6555146069578063c5292c67146071575b600080fd5b60516077565b60408051918252519081900360200190f35b6051607f565b606f6083565b005b60516089565b600054420390

### require 테스트

In [23]:
%%writefile src/TimerImportTest.js
const my = require('./Timer');   //importing a javascript file
console.log("- compiled results: " +my);
console.log("- ABI: "+my._compiled.contracts['src/Timer.sol:Timer'].abi);
console.log("- Bytecode: "+my._compiled.contracts['src/Timer.sol:Timer'].bin);

Overwriting src/TimerImportTest.js


In [24]:
!node src\TimerImportTest.js

- compiled results: [object Object]
- ABI: [{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]
- Bytecode: 608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033


### 3. 배포

geth로 배포

### node로 배포

In [25]:
%%writefile src/TimerDeployAbiBinFromFile.js
var Web3=require('web3');
var my = require('./Timer');  //cotaining abi,bin

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}


//abi from Timer.js
var _abiArray=JSON.parse(my._compiled.contracts['src/Timer.sol:Timer'].abi);   //JSON parsing needed!!
console.log("- ABI: "+my._compiled.contracts['src/Timer.sol:Timer'].abi);

//bin from Timer.js
var _bin=my._compiled.contracts['src/Timer.sol:Timer'].bin;
console.log("- Bytecode: "+my._compiled.contracts['src/Timer.sol:Timer'].bin);

var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');

_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x03D8F3F2a4Ca98C225168DF094B96347B6Eb8A70", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log("- Contract Address: "+newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/TimerDeployAbiBinFromFile.js


In [26]:
!node src/TimerDeployAbiBinFromFile.js

- ABI: [{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getStart","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]
- Bytecode: 608060405234801561001057600080fd5b5060c58061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060465760003560e01c8063b445425314604b578063bbe4fd50146063578063be9a6555146069578063c5292c67146071575b600080fd5b60516077565b60408051918252519081900360200190f35b6051607f565b606f6083565b005b60516089565b600054420390565b4290565b42600055565b6000549056fea26469706673582212208a8236282be1137b0a19a30fc3424e8a3c6b2ed0afeacf28eb506c68fdfc3c3964736f6c63430006040033
- Contra

## 4. 사용

In [48]:

%%writefile src/timerUse.js
var Web3=require('web3');
var my = require('./Timer');  //cotaining abi,bin
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

// abi from Timer.js
var _abiArray=JSON.parse(my._compiled.contracts['src/Timer.sol:Timer'].abi);   //JSON parsing needed!!
// contract address
var timer = new web3.eth.Contract(_abiArray,"0x0583A82C585a4aAF32dE71863aA9e15Ad8C01Ad0");

//timer.methods.start().send({from:"0x96c60339b39C8Faf126540B6a9BEa830fE5B43ff",gas:100000});
timer.methods.getStart().call().then(function(value) {console.log("startTime: " + value);});
timer.methods.getNow().call().then(function(value) {console.log("now: " + value);});
timer.methods.timePassed().call().then(function(value) {console.log("passed: " + value);});

Overwriting src/timerUse.js


In [56]:
!node src/timerUse.js

startTime: 1619428940
now: 1619429032
passed: 92


### geth 사용